# Product_Eshop Table Generation : Example use case Amazon sales dataset

In [58]:
import pandas as pd
import numpy as np
import kagglehub
import os

In [59]:
#download the latest version locally and return the folder path
dataset_folder = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset")

In [60]:
# List all files in the downloaded dataset folder
for root, dirs, files in os.walk(dataset_folder):
    for file in files:
        if file.endswith('.csv'):
            csv_file = os.path.join(root, file)
            print(f"Found CSV: {csv_file}")

Found CSV: C:\Users\Lenovo\.cache\kagglehub\datasets\karkavelrajaj\amazon-sales-dataset\versions\1\amazon.csv


In [61]:
# Path to save the CSV
save_path = os.path.join('../data/seed', 'amazon_sales_seed.csv')

# Load and save
pd.read_csv(csv_file).to_csv(save_path, index=False)
print(f"Dataset saved to {save_path}")

Dataset saved to ../data/seed\amazon_sales_seed.csv


In [62]:
df_amazon_seed = pd.read_csv('../data/seed/amazon_sales_seed.csv')

In [63]:
# print all columns with their types
print("All columns and dtypes:\n")
for col, dtype in zip(df_amazon_seed.columns, df_amazon_seed.dtypes):
    print(f"{col:20} : {dtype}")

All columns and dtypes:

product_id           : object
product_name         : object
category             : object
discounted_price     : object
actual_price         : object
discount_percentage  : object
rating               : object
rating_count         : object
about_product        : object
user_id              : object
user_name            : object
review_id            : object
review_title         : object
review_content       : object
img_link             : object
product_link         : object


In [64]:
# drop unnecessary columns (user_id, user_name, img_link, product_link)
columns_to_drop = ['user_id', 'user_name', 'img_link', 'product_link']
df_amazon_seed = df_amazon_seed.drop(columns=columns_to_drop,
                                     errors='ignore')

In [65]:
#check the remaining columns
print(df_amazon_seed.columns)
print(df_amazon_seed.head(2))

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'review_id', 'review_title', 'review_content'],
      dtype='object')
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   

  actual_price discount_percentage rating rating_count  \
0       ₹1,099                 64%    4.2       24,269   
1         ₹349                 43%    4.0       43,994   

                                       about_product  \
0  High Compatibility : Compatible With iPhone 12...   
1  Compatible with all Type C enabled devices, be... 

# Prepare data to model discount predictions

Type conversion

In [66]:
#Convert numeric columns
df_amazon_seed["discounted_price"] = df_amazon_seed["discounted_price"].str.replace("₹","").str.replace(",","").astype(float)
df_amazon_seed["actual_price"] = df_amazon_seed["actual_price"].str.replace("₹","").str.replace(",","").astype(float)
df_amazon_seed["discount_percentage"] = df_amazon_seed["discount_percentage"].str.replace("%","").astype(float)
df_amazon_seed["rating"] = pd.to_numeric(df_amazon_seed["rating"], errors="coerce")
df_amazon_seed["rating_count"] = pd.to_numeric(df_amazon_seed["rating_count"].str.replace(",",""), errors="coerce")

In [67]:
#Transorm the categorical column to categorical type
df_amazon_seed['category'] = df_amazon_seed['category'].astype('category')

In [68]:
df_amazon_seed['product_name'] = df_amazon_seed['product_name'].astype('string')
df_amazon_seed['about_product'] = df_amazon_seed['about_product'].astype('string')
df_amazon_seed['review_title'] = df_amazon_seed['review_title'].astype('string')
df_amazon_seed['review_content'] = df_amazon_seed['review_content'].astype('string')

In [69]:
print(df_amazon_seed.dtypes)
df_amazon_seed.head()

product_id                     object
product_name           string[python]
category                     category
discounted_price              float64
actual_price                  float64
discount_percentage           float64
rating                        float64
rating_count                  float64
about_product          string[python]
review_id                      object
review_title           string[python]
review_content         string[python]
dtype: object


,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,review_id,review_title,review_content
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,64.0,4.2,24269.0,High Compatibility : Compatible With iPhone 12...,"R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,43.0,4.0,43994.0,"Compatible with all Type C enabled devices, be...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,90.0,3.9,7928.0,【 Fast Charger& Data Sync】-With built-in safet...,"R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a..."
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,53.0,4.2,94363.0,The boAt Deuce USB 300 2 in 1 cable is compati...,"R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou..."
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,61.0,4.2,16905.0,[CHARGE & SYNC FUNCTION]- This cable comes wit...,"R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th..."


In [70]:
numeric_cols = df_amazon_seed.select_dtypes(include=['float64', 'int64']).columns
string_cols = df_amazon_seed.select_dtypes(include=['object', 'string']).columns

Handle missing values

In [71]:
# Missing numeric values
print("Missing numeric values:")
print(df_amazon_seed[numeric_cols].isna().sum())

# Missing string/text values
print("\nMissing string/text values:")
print(df_amazon_seed[string_cols].isna().sum())

Missing numeric values:
discounted_price       0
actual_price           0
discount_percentage    0
rating                 1
rating_count           2
dtype: int64

Missing string/text values:
product_id        0
product_name      0
about_product     0
review_id         0
review_title      0
review_content    0
dtype: int64


In [72]:
df_amazon_seed['rating'] = df_amazon_seed['rating'].fillna(df_amazon_seed['rating'].mean())
df_amazon_seed['rating_count'] = df_amazon_seed['rating_count'].fillna(df_amazon_seed['rating_count'].median())

In [73]:
df_amazon_seed[['rating','rating_count']].isna().sum()

rating          0
rating_count    0
dtype: int64

Generate Random data to enriche the table

In [74]:
# stock quantity between 0 and 200 randomized
n_products = len(df_amazon_seed)
df_amazon_seed['stock'] = np.random.randint(0, 200, n_products)

In [75]:
#creation date is within the last 5 years
df_amazon_seed['creation_date'] = pd.to_datetime(
    np.random.choice(pd.date_range('2020-01-01', '2025-10-01'), n_products)
)

In [76]:
#last update date is within the last year
df_amazon_seed['last_updated_date'] = df_amazon_seed['creation_date'] + pd.to_timedelta(
    np.random.randint(0, 365, n_products), unit='d'
)

In [77]:
# status is either discontinued , active or draft
df_amazon_seed['status'] = np.random.choice(['active', 'discontinued', 'draft'], n_products, p=[0.8, 0.15, 0.05])

In [78]:
# suppliers are added with their TMF is the format NNNNNN / Y / Z / T / NNN

# TVA situation possible
tva_options = ['A', 'B', 'P', 'F', 'N']
# Category of activity possible
cat_options = ['M', 'C', 'P', 'N']

def generate_supplier_id():
    identifiant = str(np.random.randint(100000, 999999))  # NNNNNN
    cle_controle = str(np.random.randint(0, 9))  # Y
    tva = np.random.choice(tva_options)  # Z
    categorie = np.random.choice(cat_options)  # T
    serie = str(np.random.randint(0, 999)).zfill(3)  # NNN

    return f"{identifiant}/{cle_controle}/{tva}/{categorie}/{serie}"

In [79]:
# Apply for all crm
df_amazon_seed['supplier_id'] = [generate_supplier_id() for _ in range(n_products)]
print(df_amazon_seed.head(2))

   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   

                                            category  discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             399.0   
1  Computers&Accessories|Accessories&Peripherals|...             199.0   

   actual_price  discount_percentage  rating  rating_count  \
0        1099.0                 64.0     4.2       24269.0   
1         349.0                 43.0     4.0       43994.0   

                                       about_product  \
0  High Compatibility : Compatible With iPhone 12...   
1  Compatible with all Type C enabled devices, be...   

                                           review_id  \
0  R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...   
1  RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...   

                                        review_

In [80]:
# df_amazon_seed after having all columns
df_amazon_seed.to_csv('../data/processed/products.csv', index=False)